In [1]:
import numpy as np
import math

In [2]:
def f(x, a):
    return a / x - np.log(x)

In [3]:
def find_x(a, eps = 1e-8):
    x = 1.5 
    for _ in range(100):
        fx = a/x - math.log(x)
        dfx = -a/(x**2) - 1/x  # Производная f'(x)
        
        if abs(dfx) < 1e-12:  # Защита от деления на ноль
            break
            
        delta = fx / dfx
        x -= delta
        
        if abs(delta) < eps:  # Проверка точности
            break
    return x

Метод Рунге-Ромберга

In [4]:
def runge_romberg_method(a_values):
    roots = []
    for a in a_values:
        root = find_x(a, eps = 1e-8)
        roots.append(root)
    
    h = 0.01
    idx = 20  # Индекс a=1.0 в списке a_values
    dx_da = (-roots[idx+2] + 8*roots[idx+1] - 8*roots[idx-1] + roots[idx-2]) / (12 * h)
    return roots, dx_da

In [5]:
a_values = [0.8 + 0.01*i for i in range(41)] 

In [6]:
roots_rr, dx_da_rr = runge_romberg_method(a_values)
print(f"Метод Рунге-Ромберга: dx*/da при a=1 равно {dx_da_rr:.6f}\n")

Метод Рунге-Ромберга: dx*/da при a=1 равно 0.638104



Полиномиальная интерполяция

In [7]:
def polynomial_interpolation(a_values):

    roots = []
    for a in a_values:
        root = find_x(a, eps = 1e-8)
        roots.append(root)
    
    # Формируем матрицу Вандермонда для полинома 2-й степени
    n = len(a_values)
    V = np.zeros((n, 3))
    for i in range(n):
        V[i][0] = a_values[i]**2
        V[i][1] = a_values[i]
        V[i][2] = 1
    
    # Решаем систему V^T * V * coeff = V^T * roots
    VT = V.T
    A = VT @ V
    b = VT @ roots
    coeff = np.linalg.solve(A, b)
    
    # Вычисляем производную полинома: 2*c0*a + c1
    poly_deriv = [2*coeff[0], coeff[1]]
    
    # Вычисляем значение производной при a=1
    dx_da = poly_deriv[0] * 1 + poly_deriv[1]
    
    return dx_da, coeff

In [8]:
a_values = [0.8 + 0.01*i for i in range(41)] 

In [9]:
dx_da_poly, poly_coeff = polynomial_interpolation(a_values)
print(f"Метод полиномиальной интерполяции:\ndx*/da при a=1 равно {dx_da_poly:.8f}")
print(f"Коэффициенты полинома: {poly_coeff}")

Метод полиномиальной интерполяции:
dx*/da при a=1 равно 0.63876046
Коэффициенты полинома: [-0.07410049  0.78696144  1.05036366]


Метод регуляризации

In [10]:
def regularization_method(a_values):
    roots = []
    for a in a_values:
        root = find_x(a, eps = 1e-8)
        roots.append(root)
    
    a_target = 1.0
    nearest_idx = np.argmin(np.abs(np.array(a_values) - a_target))
    window = slice(max(0, nearest_idx-2), min(len(a_values), nearest_idx+3))
    
    a_window = np.array(a_values)[window]
    x_window = np.array(roots)[window]
    
    a_mean = np.mean(a_window)
    x_mean = np.mean(x_window)
    
    S_xy = np.sum((a_window - a_mean) * (x_window - x_mean))
    S_xx = np.sum((a_window - a_mean)**2)
    
    return S_xy / S_xx


In [11]:
dx_da_reg = regularization_method(a_values)
print(f"Метод регуляризации:\ndx*/da при a=1 равно {dx_da_reg:.8f}")


Метод регуляризации:
dx*/da при a=1 равно 0.63811255


In [12]:
estimates = [dx_da_rr, dx_da_poly, dx_da_reg]
max_diff = max(estimates) - min(estimates)
print(f"\nМаксимальная разница между методами: {max_diff:.2e}")


Максимальная разница между методами: 6.57e-04
